In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import math
from sklearn.model_selection import train_test_split
import numpy as np 
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../.."))

from config.model_config import ModelConfig
from config.experiment_config import ExperimentConfig, AvailableDatasets
from vol_predict.features.preprocessor import OneToOnePreprocessor

from vol_predict.models.dl.mlp_predictor import MLPPredictor as Model
from vol_predict.models.baselines.naive_predictor import NaivePredictor as Baseline
from vol_predict.loss.tm_loss import MixtureNormalNLL, MixtureLogNormalNLL, MixtureWeibullNLL, HingeNormalMixtureNLL, MixtureInverseGaussianNLL
from vol_predict.models.mixture.tm_pred2 import TM_N_Predictor,TM_LN_Predictor,TM_IG_Predictor,TM_W_Predictor

from vol_predict.loss.loss import Loss

from run import initialize_runner, run_backtest

In [40]:
data = pd.read_csv("../../data:/data_df.csv")
data_t1 = data[data.columns[0:13]]
features_data = data_t1[data_t1.columns[3:13]].values.astype(np.float32)     
vol = data_t1['vol'].values.astype(np.float32)     

In [38]:
num_samples = len(vol)
train_window = 300  
test_window = 50
num_iterations = (num_samples - train_window) // test_window

In [8]:
model_config = ModelConfig(
    lr=1e-3,
    hidden_size=64,         # not used by this model, but needed for compatibility
    n_layers=2,             # same as above
    n_epochs=20,
    n_features=10,          # total flattened features (5x2)
    optimizer=torch.optim.Adam,
    weights_decay=1e-4,
    batch_size=32,
    loss=Loss.TM_N_NLL,   # must be set to your MixtureNormalNLL loss enum
    metrics=(torch.nn.MSELoss(),),
    vol_calc_method=VolatilityMethod("squared_returns")
)

NameError: name 'VolatilityMethod' is not defined